In [1]:
import psycopg2
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
from matplotlib import pyplot as plt
warnings.simplefilter("ignore", UserWarning)

In [2]:
#pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [5]:
print(user)

natalia


In [6]:
def connect_to_redshift(dbname, host, port, user, password):
    """Method that connects to redshift. This gives a warning so will look for another solution"""

    connect = psycopg2.connect(
        dbname=dbname, host=host, port=port, user=user, password=password
    )

    print("connection to redshift made")

    return connect

In [7]:
connect=connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [8]:
query="""select otc.*
              
         from bootcamp.online_transactions_cleaned otc   
      
      """

In [9]:
otc=pd.read_sql(query,connect)

In [10]:
otc.head(10)

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536384,22189,CREAM HEART CARD HOLDER,3.95,4,15.80,2010-12-01 09:53:00,u18074,United Kingdom
1,536392,21891,TRADITIONAL WOODEN SKIPPING ROPE,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
2,536401,22068,BLACK PIRATE TREASURE CHEST,1.65,2,3.30,2010-12-01 11:21:00,u15862,United Kingdom
3,536412,22961,JAM MAKING SET PRINTED,1.45,24,34.80,2010-12-01 11:49:00,u1792,United Kingdom
4,536420,21889,WOODEN BOX OF DOMINOES,1.25,12,15.00,2010-12-01 12:03:00,u16583,United Kingdom
5,536464,22742,MAKE YOUR OWN PLAYTIME CARD KIT,2.95,1,2.95,2010-12-01 12:23:00,u17968,United Kingdom
6,536520,22557,PLASTERS IN TIN VINTAGE PAISLEY,1.65,2,3.30,2010-12-01 12:43:00,u14729,United Kingdom
7,536525,20973,12 PENCIL SMALL TUBE WOODLAND,0.65,2,1.30,2010-12-01 12:54:00,u14078,United Kingdom
8,536378,84519A,TOMATO CHARLIE+LOLA COASTER SET,2.95,6,17.70,2010-12-01 09:37:00,u14688,United Kingdom
9,536401,22100,SKULLS SQUARE TISSUE BOX,1.25,2,2.50,2010-12-01 11:21:00,u15862,United Kingdom


In [19]:
otc.shape

(399841, 9)

In [11]:
otc.stock_code.nunique()

3679

In [12]:
print("First transaction: {}, Last transaction: {}".format(min(otc.invoice_date),max(otc.invoice_date)))

First transaction: 2010-12-01 08:26:00, Last transaction: 2011-12-09 12:50:00


# Quality check

From the initial otc table with 399841 records the following records were dropped:

1. 4026 records cancellation pairs 
2. 34 records with zero prices
3. 153 records with invoices to beginn with "C","D","P" letters. The particular invoices belong probably to expenses.

The new cleaned table has 395628 records.

## Negative quantities
### 8507 records with negative quantities


In [20]:
otc_negative=otc[otc.total_order_value<0]
otc_negative

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
3,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,1.65,-12,-19.80,2010-12-01 10:24:00,u17548,United Kingdom
102,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,0.29,-24,-6.96,2010-12-01 10:24:00,u17548,United Kingdom
200,C536548,22631,CIRCUS PARADE LUNCH BOX,1.95,-1,-1.95,2010-12-01 14:33:00,u12472,Germany
253,C536548,22580,ADVENT CALENDAR GINGHAM SACK,5.95,-4,-23.80,2010-12-01 14:33:00,u12472,Germany
429,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,4.65,-1,-4.65,2010-12-01 09:49:00,u15311,United Kingdom
...,...,...,...,...,...,...,...,...,...
399741,C580919,23486,ANTIQUE HEART SHELF UNIT,16.65,-2,-33.30,2011-12-06 14:07:00,u16571,United Kingdom
399793,C581229,23158,SET OF 5 LUCKY CAT MAGNETS,2.08,-36,-74.88,2011-12-08 10:14:00,u12558,USA
399812,C581409,22364,GLASS JAR DIGESTIVE BISCUITS,2.95,-2,-5.90,2011-12-08 14:08:00,u12476,Germany
399827,C581466,21535,RED RETROSPOT SMALL MILK JUG,2.55,-2,-5.10,2011-12-08 19:20:00,u13883,United Kingdom


In [21]:
c_pairs=otc.groupby([otc.customer_id, otc.stock_code, otc.quantity.abs()])\
            .filter(lambda x: (len(x.quantity.abs()) % 2 == 0) and (x.quantity.sum() == 0))\
            .sort_values(by=['customer_id', 'stock_code'])

### From 8507 records with negative quantities 4026 are cancellation pairs. The particular transactions will not be included, as they correspond to cancellation of orders or returns.

In [22]:
c_pairs

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
105758,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.6,2011-01-18 10:17:00,u12346,United Kingdom
110049,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,74215,77183.6,2011-01-18 10:01:00,u12346,United Kingdom
110347,543370,22839,3 TIER CAKE TIN GREEN AND CREAM,14.95,2,29.9,2011-02-07 14:51:00,u12359,Cyprus
140932,C549955,22839,3 TIER CAKE TIN GREEN AND CREAM,14.95,-2,-29.9,2011-04-13 13:38:00,u12359,Cyprus
382817,573173,22941,CHRISTMAS LIGHTS 10 REINDEER,8.50,2,17.0,2011-10-28 10:10:00,u12362,Belgium
...,...,...,...,...,...,...,...,...,...
278390,C577832,23245,SET OF 3 REGENCY CAKE TINS,4.95,-4,-19.8,2011-11-22 10:18:00,u18274,United Kingdom
255594,575485,84509A,SET OF 4 ENGLISH ROSE PLACEMATS,3.75,4,15.0,2011-11-09 17:03:00,u18274,United Kingdom
345949,C577832,84509A,SET OF 4 ENGLISH ROSE PLACEMATS,3.75,-4,-15.0,2011-11-22 10:18:00,u18274,United Kingdom
292508,C577832,84988,SET OF 72 PINK HEART PAPER DOILIES,1.45,-12,-17.4,2011-11-22 10:18:00,u18274,United Kingdom


In [23]:
c_pairs_df=c_pairs.index.to_frame().reset_index()
c_pairs_df

,index,0
0,105758,105758
1,110049,110049
2,110347,110347
3,140932,140932
4,382817,382817
...,...,...
4021,278390,278390
4022,255594,255594
4023,345949,345949
4024,292508,292508


In [24]:
c_pairs_df = c_pairs_df.drop(columns=c_pairs_df.columns[1])

In [25]:
c_pairs_df

,index
0,105758
1,110049
2,110347
3,140932
4,382817
...,...
4021,278390
4022,255594
4023,345949
4024,292508


In [26]:
otc_cleaned = otc.drop(labels=c_pairs.index, axis=0)

From the initial otc table with 399841 records the 4026 were dropped. The new table has 395815 records.

In [27]:
otc_cleaned

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536378,21929,JUMBO BAG PINK VINTAGE PAISLEY,1.95,10,19.50,2010-12-01 09:37:00,u14688,United Kingdom
1,536384,22469,HEART OF WICKER SMALL,1.45,40,58.00,2010-12-01 09:53:00,u18074,United Kingdom
2,536389,22193,RED DINER WALL CLOCK,8.50,2,17.00,2010-12-01 10:03:00,u12431,Australia
3,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,1.65,-12,-19.80,2010-12-01 10:24:00,u17548,United Kingdom
4,536395,22727,ALARM CLOCK BAKELIKE RED,3.75,8,30.00,2010-12-01 10:47:00,u13767,United Kingdom
...,...,...,...,...,...,...,...,...,...
399836,581495,15056N,EDWARDIAN PARASOL NATURAL,5.95,36,214.20,2011-12-09 10:20:00,u14051,United Kingdom
399837,581502,23103,JINGLE BELL HEART DECORATION,1.65,8,13.20,2011-12-09 10:51:00,u1591,United Kingdom
399838,581538,21328,BALLOONS WRITING SET,1.65,1,1.65,2011-12-09 11:34:00,u14446,United Kingdom
399839,581574,21258,VICTORIAN SEWING BOX LARGE,12.75,2,25.50,2011-12-09 12:09:00,u12526,Germany


In [69]:
#399841-4026=395815


# Zero prices will be removed from the dataset.
There are 34 records with zero prices

In [28]:
otc_zero_price=otc_cleaned[otc.price==0]
otc_zero_price

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
7543,540372,22553,PLASTERS IN TIN SKULLS,0.0,24,0.0,2011-01-06 16:41:00,u13081,United Kingdom
9191,537197,22841,ROUND CAKE TIN VINTAGE GREEN,0.0,1,0.0,2010-12-05 14:02:00,u12647,Germany
14130,539722,22423,REGENCY CAKESTAND 3 TIER,0.0,10,0.0,2010-12-21 13:45:00,u14911,EIRE
21712,540372,22090,PAPER BUNTING RETROSPOT,0.0,24,0.0,2011-01-06 16:41:00,u13081,United Kingdom
32697,550188,22636,CHILDS BREAKFAST SET CIRCUS PARADE,0.0,1,0.0,2011-04-14 18:57:00,u12457,Switzerland
49921,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,0.0,16,0.0,2011-02-10 13:08:00,u1756,United Kingdom
56381,548871,22162,HEART GARLAND RUSTIC PADDED,0.0,2,0.0,2011-04-04 14:42:00,u1441,United Kingdom
78615,548318,22055,MINI CAKE STAND HANGING STRAWBERY,0.0,5,0.0,2011-03-30 12:45:00,u13113,United Kingdom
89953,553000,47566,PARTY BUNTING,0.0,4,0.0,2011-05-12 15:21:00,u17667,United Kingdom
90579,539263,22580,ADVENT CALENDAR GINGHAM SACK,0.0,4,0.0,2010-12-16 14:36:00,u1656,United Kingdom


In [29]:
otc_zero_price.shape


(34, 9)

In [30]:
otc_zero_price.index

Index([  7543,   9191,  14130,  21712,  32697,  49921,  56381,  78615,  89953,
        90579, 100766, 111492, 152700, 164135, 166795, 217105, 220402, 220632,
       221638, 241390, 243388, 246700, 259693, 290488, 310230, 310599, 320530,
       343313, 360218, 361213, 366547, 374420, 379172, 391944],
      dtype='int64')

In [31]:
otc_cleaned_final=otc_cleaned.drop(labels=otc_zero_price.index,axis=0)

In [32]:
otc_cleaned_final.shape


(395781, 9)

# Invoices with "C","D","P" will be not included, as they correspond to the expenses or costs of the company

In [33]:
CDP_stock_code=otc_cleaned_final[otc_cleaned_final.stock_code.str[0].isin(["C","D","P"])]
CDP_stock_code

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
8293,537378,C2,CARRIAGE,50.00,1,50.00,2010-12-06 13:06:00,u14911,EIRE
13988,C540164,C2,CARRIAGE,50.00,-1,-50.00,2011-01-05 12:02:00,u14911,EIRE
17107,536540,C2,CARRIAGE,50.00,1,50.00,2010-12-01 14:05:00,u14911,EIRE
21442,538002,C2,CARRIAGE,50.00,1,50.00,2010-12-09 11:48:00,u14932,Channel Islands
22257,539421,C2,CARRIAGE,50.00,1,50.00,2010-12-17 14:21:00,u14016,EIRE
...,...,...,...,...,...,...,...,...,...
360624,569246,DOT,DOTCOM POSTAGE,490.25,1,490.25,2011-10-03 09:57:00,u14096,United Kingdom
362195,565383,DOT,DOTCOM POSTAGE,16.46,1,16.46,2011-09-02 15:45:00,u14096,United Kingdom
371671,571441,DOT,DOTCOM POSTAGE,743.62,1,743.62,2011-10-17 13:31:00,u14096,United Kingdom
379609,575607,DOT,DOTCOM POSTAGE,908.16,1,908.16,2011-11-10 12:37:00,u14096,United Kingdom


In [34]:
CDP_stock_code.index

Index([  8293,  13988,  17107,  21442,  22257,  25793,  26523,  27220,  29902,
        33341,
       ...
       336537, 341537, 341715, 351648, 358252, 360624, 362195, 371671, 379609,
       387036],
      dtype='int64', length=153)

In [35]:
otc_final=otc_cleaned_final.drop(labels=CDP_stock_code.index,axis=0)
otc_final

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536378,21929,JUMBO BAG PINK VINTAGE PAISLEY,1.95,10,19.50,2010-12-01 09:37:00,u14688,United Kingdom
1,536384,22469,HEART OF WICKER SMALL,1.45,40,58.00,2010-12-01 09:53:00,u18074,United Kingdom
2,536389,22193,RED DINER WALL CLOCK,8.50,2,17.00,2010-12-01 10:03:00,u12431,Australia
3,C536391,22557,PLASTERS IN TIN VINTAGE PAISLEY,1.65,-12,-19.80,2010-12-01 10:24:00,u17548,United Kingdom
4,536395,22727,ALARM CLOCK BAKELIKE RED,3.75,8,30.00,2010-12-01 10:47:00,u13767,United Kingdom
...,...,...,...,...,...,...,...,...,...
399836,581495,15056N,EDWARDIAN PARASOL NATURAL,5.95,36,214.20,2011-12-09 10:20:00,u14051,United Kingdom
399837,581502,23103,JINGLE BELL HEART DECORATION,1.65,8,13.20,2011-12-09 10:51:00,u1591,United Kingdom
399838,581538,21328,BALLOONS WRITING SET,1.65,1,1.65,2011-12-09 11:34:00,u14446,United Kingdom
399839,581574,21258,VICTORIAN SEWING BOX LARGE,12.75,2,25.50,2011-12-09 12:09:00,u12526,Germany


In [124]:
# 395781-153=395628


## Load 'otc_final' to csv file for the futher analysis

In [40]:
otc_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395628 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            395628 non-null  object        
 1   stock_code         395628 non-null  object        
 2   description        395628 non-null  object        
 3   price              395628 non-null  float64       
 4   quantity           395628 non-null  int64         
 5   total_order_value  395628 non-null  float64       
 6   invoice_date       395628 non-null  datetime64[ns]
 7   customer_id        395628 non-null  object        
 8   country            395628 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 30.2+ MB


In [39]:
otc_final.to_csv("./data/otc_final.csv",index=False)

# Quering the cleaned data


In [276]:
otc_cleaned.head()

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536370,21791,VINTAGE HEADS AND TAILS CARD GAME,1.25,24,30.00,2010-12-01 08:45:00,u12583,France
1,536381,15056BL,EDWARDIAN PARASOL BLACK,5.95,2,11.90,2010-12-01 09:41:00,u15311,United Kingdom
2,536385,22783,SET 3 WICKER OVAL BASKETS W LIDS,19.95,1,19.95,2010-12-01 09:56:00,u1742,United Kingdom
3,536390,22968,ROSE COTTAGE KEEPSAKE BOX,8.50,8,68.00,2010-12-01 10:19:00,u17511,United Kingdom
4,536392,22128,PARTY CONES CANDY ASSORTED,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom


In [277]:
otc_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395815 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            395815 non-null  object        
 1   stock_code         395815 non-null  object        
 2   description        395815 non-null  object        
 3   price              395815 non-null  float64       
 4   quantity           395815 non-null  int64         
 5   total_order_value  395815 non-null  float64       
 6   invoice_date       395815 non-null  datetime64[ns]
 7   customer_id        395815 non-null  object        
 8   country            395815 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 38.3+ MB


In [278]:
otc_cleaned.describe()

,price,quantity,total_order_value,invoice_date
count,395815.000000,395815.000000,395815.000000,395815
mean,2.922664,12.348698,20.926967,2011-07-10 15:03:14.760229376
min,0.000000,-9360.000000,-6539.400000,2010-12-01 08:26:00
25%,1.250000,2.000000,4.250000,2011-04-07 09:16:00
50%,1.950000,5.000000,11.700000,2011-07-31 11:45:00
75%,3.750000,12.000000,19.680000,2011-10-20 12:31:00
max,1599.260000,12540.000000,38970.000000,2011-12-09 12:50:00
std,7.169738,50.089251,95.835420,NaN


In [279]:
otc_final.country.nunique()

37

In [280]:
otc_final.stock_code.nunique()

3664

In [281]:
otc_final.description.nunique()

3629

In [283]:
otc_final.customer_id.nunique()

4351

## How many invoices does the table contain?


In [136]:
query="""select count(distinct invoice) as total_num_invoices
              
         from bootcamp.online_transactions_cleaned otc  
      
      """

In [137]:
num_inv=pd.read_sql(query,connect)

In [138]:
num_inv

,total_num_invoices
0,21791


## When was the first and last purchase?


In [142]:
query=""" select 
                 min(invoice_date) as first_purchase,              
                 max(invoice_date) as last_purchase
         from    bootcamp.online_transactions_cleaned otc 
         group by invoice
         order by                

"""

In [143]:
first_last_purchase=pd.read_sql(query,connect)

In [144]:
first_last_purchase


,first_purchase,last_purchase
0,2010-12-01 08:26:00,2011-12-09 12:50:00


In [158]:
query=""" with cte as (select invoice,
          description,
          invoice_date as first_purchase,
          sum(quantity) as total_quantity
         from    bootcamp.online_transactions_cleaned otc 
         group by invoice,description,invoice_date
         order by invoice_date)
         
         select invoice,
                description,
                first_purchase
         from cte
         limit 1;

"""

In [148]:
question2=pd.read_sql(query,connect)

In [149]:
question2

,invoice,description,invoice_date,total_quantity
0,536365,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00,6
1,536365,CREAM CUPID HEARTS COAT HANGER,2010-12-01 08:26:00,8
2,536365,RED WOOLLY HOTTIE WHITE HEART.,2010-12-01 08:26:00,6
3,536365,SET 7 BABUSHKA NESTING BOXES,2010-12-01 08:26:00,2
4,536365,CREAM HANGING HEART T-LIGHT HOLDER,2010-12-01 08:26:00,6
...,...,...,...,...
394664,581587,CHILDREN'S APRON DOLLY GIRL,2011-12-09 12:50:00,6
394665,581587,PLASTERS IN TIN STRONGMAN,2011-12-09 12:50:00,12
394666,581587,CHILDRENS APRON SPACEBOY DESIGN,2011-12-09 12:50:00,8
394667,581587,PACK OF 20 SPACEBOY NAPKINS,2011-12-09 12:50:00,12


In [159]:
question2=pd.read_sql(query,connect)
question2

,invoice,description,first_purchase
0,536365,KNITTED UNION FLAG HOT WATER BOTTLE,2010-12-01 08:26:00


In [162]:
query=""" with cte as (select invoice,
          description,
          invoice_date as last_purchase,
          sum(quantity) as total_quantity
         from    bootcamp.online_transactions_cleaned otc 
         group by invoice,description,invoice_date
         order by invoice_date)
         
         select invoice,
                description,
                last_purchase
         from cte
         order by last_purchase desc
         limit 1;

"""

In [163]:
question2=pd.read_sql(query,connect)
question2

,invoice,description,last_purchase
0,581587,ALARM CLOCK BAKELIKE RED,2011-12-09 12:50:00


## How many customers does the table contain?


In [164]:
query=""" select count(distinct customer_id) as numberOfCustomers
          from bootcamp.online_transactions_cleaned otc 
"""

In [165]:
# otc.customer_id.nunique()
question3=pd.read_sql(query,connect)
question3

,numberofcustomers
0,4363


## How many different types of stocks did the customer purchase?


In [179]:
query="""
         select customer_id,
                count(distinct stock_code) as numOfStocks    
         from bootcamp.online_transactions_cleaned otc  
         group by customer_id
         order by numOfStocks desc
      
      """

In [180]:
question4=pd.read_sql(query,connect)
question4

,customer_id,numofstocks
0,u14911,1792
1,u12748,1767
2,u17841,1330
3,u14096,1119
4,u14298,884
...,...,...
4358,u1699,1
4359,u13302,1
4360,u15668,1
4361,u16462,1


In [183]:
# The total number of distinct stock items in the customer purchases
query="""select count(distinct stock_code) as total_number_stocks
from bootcamp.online_transactions_cleaned otc 
"""

In [184]:
question4_2=pd.read_sql(query,connect)
question4_2

,total_number_stocks
0,3679


## What is the most popular stock? (Considering top 10 sold items - How does this differ across markets?)


###  Top 10 sold items

In [44]:
otc.stock_code.nunique()

3679

In [100]:
query="""
         select  
                 description,
                 stock_code as top_item,
                 count(stock_code) as frequency 
         from bootcamp.online_transactions_cleaned otc  
         group by top_item,description
         order by frequency  desc
         limit 10;
         
      
      """

In [101]:
top10f=pd.read_sql(query,connect)
top10f

,description,top_item,frequency
0,CREAM HANGING HEART T-LIGHT HOLDER,85123A,2065
1,REGENCY CAKESTAND 3 TIER,22423,1894
2,JUMBO BAG RED RETROSPOT,85099B,1659
3,PARTY BUNTING,47566,1409
4,ASSORTED COLOUR BIRD ORNAMENT,84879,1405
5,LUNCH BAG RED RETROSPOT,20725,1346
6,SET OF 3 CAKE TINS PANTRY DESIGN,22720,1224
7,POPCORN HOLDER,22197,1110
8,JUMBO BAG VINTAGE DOILEY,23203,1107
9,LUNCH BAG BLACK SKULL.,20727,1099


In [ ]:
# Top 10 sold items aggregating for all countries

In [107]:
query="""
         SELECT description,
                stock_code,
                SUM(quantity) AS total_quantity
         FROM  bootcamp.online_transactions_cleaned
         GROUP BY description, stock_code
         ORDER BY total_quantity DESC
         LIMIT 10;
      
      """

In [108]:
top10m=pd.read_sql(query,connect)
top10m

,description,stock_code,total_quantity
0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,84077,53119
1,POPCORN HOLDER,22197,48689
2,JUMBO BAG RED RETROSPOT,85099B,44963
3,ASSORTED COLOUR BIRD ORNAMENT,84879,35215
4,CREAM HANGING HEART T-LIGHT HOLDER,85123A,34185
5,PACK OF 72 RETROSPOT CAKE CASES,21212,33386
6,RABBIT NIGHT LIGHT,23084,27045
7,MINI PAINT SET VINTAGE,22492,25880
8,PACK OF 12 LONDON TISSUES,22616,25305
9,PACK OF 60 PINK PAISLEY CAKE CASES,21977,24129


## Top 10 sold items across markets

In [ ]:
# Top 10 stock items for each country
query=""" WITH RankedSales AS (
    SELECT 
        stock_code, 
        description,
        country, 
        SUM(quantity) AS total_sold,
        RANK() OVER(PARTITION BY country ORDER BY SUM(quantity) DESC) AS rank
    FROM bootcamp.online_transactions_cleaned
    GROUP BY description,stock_code, country
)
SELECT description,stock_code, country,rank, total_sold
FROM RankedSales
WHERE rank <= 10;
      """

In [303]:

top10mCountryRank=pd.read_sql(query,connect)
top10mCountryRank

,description,stock_code,country,rank,total_sold
0,MINI PAINT SET VINTAGE,22492,Australia,1,2916
1,RABBIT NIGHT LIGHT,23084,Australia,2,1884
2,RED HARMONICA IN BOX,21915,Australia,3,1704
3,RED TOADSTOOL LED NIGHT LIGHT,21731,Australia,4,1344
4,DOLLY GIRL LUNCH BOX,22630,Australia,5,1024
...,...,...,...,...,...
470,PACK OF 12 LONDON TISSUES,22616,United Kingdom,6,24297
471,BROCADE RING PURSE,17003,United Kingdom,7,22672
472,PACK OF 72 RETROSPOT CAKE CASES,21212,United Kingdom,8,22182
473,VICTORIAN GLASS HANGING T-LIGHT,22178,United Kingdom,9,21427


In [290]:
# Top 10 stock items across the countries
query=""" WITH RankedSales AS (
    SELECT 
        stock_code, 
        description,
        country, 
        SUM(quantity) AS total_sold,
        RANK() OVER(PARTITION BY country ORDER BY SUM(quantity) DESC) AS rank
    FROM bootcamp.online_transactions_cleaned
    GROUP BY description,stock_code, country
)
SELECT description,stock_code, country,rank, total_sold
FROM RankedSales
WHERE rank =1
order by total_sold desc
limit 10;

      """

In [291]:
top1CountryRank=pd.read_sql(query,connect)
top1CountryRank

,description,stock_code,country,rank,total_sold
0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,84077,United Kingdom,1,47886
1,RABBIT NIGHT LIGHT,23084,Netherlands,1,4801
2,RABBIT NIGHT LIGHT,23084,France,1,3999
3,RABBIT NIGHT LIGHT,23084,Japan,1,3401
4,MINI PAINT SET VINTAGE,22492,Sweden,1,2916
5,MINI PAINT SET VINTAGE,22492,Australia,1,2916
6,POPCORN HOLDER,22197,EIRE,1,1785
7,ROUND SNACK BOXES SET OF4 WOODLAND,22326,Germany,1,1206
8,CHILDRENS CUTLERY POLKADOT PINK,84997D,Spain,1,1089
9,PLASTERS IN TIN WOODLAND ANIMALS,22554,Switzerland,1,636


## What is the average order value i.e. price * quantity?


In [319]:
query=""" 
          select cast(date_part(year,invoice_date) as int) as invoice_year,
                 cast(date_part(month,invoice_date) as int) as invoice_month,
                 round(sum(total_order_value)/count(distinct invoice),2) as average_oder_value
          from bootcamp.online_transactions_cleaned
          group by invoice_year, invoice_month
          order by invoice_year,invoice_month


""" 

In [320]:
average_oder_value=pd.read_sql(query,connect)
average_oder_value

,invoice_year,invoice_month,average_oder_value
0,2010,12,324.14
1,2011,1,384.96
2,2011,2,367.67
3,2011,3,361.36
4,2011,4,310.40
5,2011,5,359.92
6,2011,6,380.27
7,2011,7,373.45
8,2011,8,407.02
9,2011,9,453.58


## How many Stocks have the Description “Unknown”?

In [249]:
query=""" 
          select count(*) as num_trans_no_Description
          from bootcamp.online_transactions_cleaned
          
          where description = 'Unknown'
          
          
          """ 

In [243]:
otc_final[otc_final.description=="Unknown"]

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
71,536557,22686,Unknown,1.25,1,1.25,2010-12-01 14:41:00,u17841,United Kingdom
633,536408,21705,Unknown,1.65,12,19.80,2010-12-01 11:41:00,u14307,United Kingdom
848,536597,21703,Unknown,0.42,4,1.68,2010-12-01 17:35:00,u18011,United Kingdom
893,536736,21703,Unknown,0.42,192,80.64,2010-12-02 12:59:00,u17381,United Kingdom
912,536500,46000M,Unknown,1.55,10,15.50,2010-12-01 12:35:00,u17377,United Kingdom
...,...,...,...,...,...,...,...,...,...
397793,579106,46000M,Unknown,1.55,6,9.30,2011-11-28 11:42:00,u16705,United Kingdom
397824,579260,23444,Unknown,15.00,1,15.00,2011-11-29 08:41:00,u14618,United Kingdom
397894,579535,23444,Unknown,15.00,1,15.00,2011-11-30 09:50:00,u15414,United Kingdom
398125,580622,23444,Unknown,15.00,1,15.00,2011-12-05 12:09:00,u17006,United Kingdom


In [250]:
no_descr=pd.read_sql(query,connect)
no_descr

,num_trans_no_description
0,1172
